In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import numpy as np
from google.colab import drive

In [ ]:
"""url = 'https://www.glassdoor.com.ar/Sueldos/data-scientist-sueldo-SRCH_KO0,14.htm' # Las url tienen esta estructura: nombre_puesto + "-sueldo" + "-SRCH_KO0," + cant_caracteres_nombre_puesto (por ej, data-scientist tiene 14 caracteres)
search = requests.get(url, verify = True, headers = {"User-Agent":'Mozilla/5.0'}) # Hacemos el request a la página para acceder a la misma
print(f'El status es: {search.status_code}') # Chequeamos que haya salido todo bien
search_parseada = bs(search.content, 'html.parser') # Parseamos el contenido del request como un html
print(search_parseada.prettify()[:20000]) # Printeamos los primeros n caracteres para ver qué onda"""

'url = \'https://www.glassdoor.com.ar/Sueldos/data-scientist-sueldo-SRCH_KO0,14.htm\' # Las url tienen esta estructura: nombre_puesto + "-sueldo" + "-SRCH_KO0," + cant_caracteres_nombre_puesto (por ej, data-scientist tiene 14 caracteres)\nsearch = requests.get(url, verify = True, headers = {"User-Agent":\'Mozilla/5.0\'}) # Hacemos el request a la página para acceder a la misma\nprint(f\'El status es: {search.status_code}\') # Chequeamos que haya salido todo bien\nsearch_parseada = bs(search.content, \'html.parser\') # Parseamos el contenido del request como un html\nprint(search_parseada.prettify()[:20000]) # Printeamos los primeros n caracteres para ver qué onda'

In [ ]:
"""tag_sueldo1 = search_parseada.find(name = 'div', attrs = {'class' : 'py css-17435dd'}) # tag padre que contiene metadata del primer sueldo
print(tag_sueldo1.prettify()[:200]) # Vemos el tag general del primer depto"""

"tag_sueldo1 = search_parseada.find(name = 'div', attrs = {'class' : 'py css-17435dd'}) # tag padre que contiene metadata del primer sueldo\nprint(tag_sueldo1.prettify()[:200]) # Vemos el tag general del primer depto"

In [ ]:
"""# Si queremos buscar todos los sueldos que aparecen en la página, usamos el find_all para la primera búsqueda que hicimos
tag_sueldos = search_parseada.findAll(name = 'div', attrs = {'class' : 'py css-17435dd'})
print(f'El tipo de objeto es: {type(tag_sueldos)}, que puede ser tratado como una lista (es decir, podemos recorrerla)')
print(f'La cantidad de puestos que encontramos es: {len(tag_sueldos)}')"""

"# Si queremos buscar todos los sueldos que aparecen en la página, usamos el find_all para la primera búsqueda que hicimos\ntag_sueldos = search_parseada.findAll(name = 'div', attrs = {'class' : 'py css-17435dd'})\nprint(f'El tipo de objeto es: {type(tag_sueldos)}, que puede ser tratado como una lista (es decir, podemos recorrerla)')\nprint(f'La cantidad de puestos que encontramos es: {len(tag_sueldos)}')"

In [ ]:
"""# Entonces, si queremos todos los urls de estos sueldos, podemos hacer la siguiente iteración:
base = 'https://www.glassdoor.com.ar' # Defino el dominio de glassdooor para agregárselo a los href de cada sueldo

url_sueldos = [base + t.find(name = 'a', attrs = {'class' : 'css-f3vw95 e1aj7ssy3'}).attrs['href'] for t in tag_sueldos] 

url_sueldos # Y así obtenemos la lsita de los primeros 20 departamentos mostrados en la primera página de glassdoor"""

"# Entonces, si queremos todos los urls de estos sueldos, podemos hacer la siguiente iteración:\nbase = 'https://www.glassdoor.com.ar' # Defino el dominio de glassdooor para agregárselo a los href de cada sueldo\n\nurl_sueldos = [base + t.find(name = 'a', attrs = {'class' : 'css-f3vw95 e1aj7ssy3'}).attrs['href'] for t in tag_sueldos] \n\nurl_sueldos # Y así obtenemos la lsita de los primeros 20 departamentos mostrados en la primera página de glassdoor"

In [ ]:
# Podemos agregarle a cada búsqueda el prefijo "junior" y "senior" para hacer data augmentation
puestos_eng = ["Technical Leader", "Project Leader", "Project Manager", "Product Owner", "Data Architect", "Software Architect","Scrum Master","Machine Learning Engineer","Data Scientist","Data Engineer","Data Analyst","Database Administrator","UX","Developer","SysAdmin","DevOps","IT Recruiter","QA", "Tester","Business Analyst","Consultant","Functional Analyst","Designer"]
puestos_esp = ["Líder técnico", "Líder de proyecto", "Gerente de proyecto", "Propietario del producto", "Arquitecto de datos", "Arquitecto de software", "Scrum Master", "Ingeniero de aprendizaje automático", "Científico de datos", "Ingeniero de datos" , "Analista de datos", "Administrador de base de datos", "UX", "Desarrollador", "SysAdmin", "DevOps", "Reclutador de TI", "Control de calidad", "Probador", "Analista comercial", "Analista de datos", "Consultor","Analista Funcional","Diseñador"]
puestos = puestos_eng+puestos_esp
# En el caso de los developers, hay varias palabras que suelen aparecer junto a ellos, podemos ver qué pasa según el lenguaje, tipo de desarrollo y framework, para eso, hagamos una lista de los más usados en la industria:
lenguajes = ["Javascript","Python","Java","COBOL","PHP",".NET","Node","React","jQuery","Angular","Vue","Go","Rust","Blockchain","Web","Fullstack","frontend","backend"]

def aumentar_developers(puestos):
  puestos_expanded = []
  for puesto in puestos:
    if puesto=="Developer":
      for lenguaje in lenguajes:
        puesto_temp = lenguaje+" Developer"
        puestos_expanded.append(puesto_temp)
    elif puesto=="Desarrollador":
      for lenguaje in lenguajes:
        puesto_temp = "Desarrollador "+lenguaje
        puestos_expanded.append(puesto_temp)
    puestos_expanded.append(puesto)
  return puestos_expanded

puestos_expanded = aumentar_developers(puestos)
#puestos_expanded

def agregar_seniority(puestos_expanded):
  lista_puestos_y_seniorities = []
  for puesto in puestos_expanded:
    jr = "junior " + puesto
    ssr = puesto
    sr = "senior " + puesto
    seniorities = [jr, ssr, sr]
    lista_puestos_y_seniorities.extend(seniorities)
  return lista_puestos_y_seniorities

lista_puestos_y_seniorities = agregar_seniority(puestos_expanded)
print(lista_puestos_y_seniorities)

# Acá procesamos cada elemento de la lista para embeberlo luego en la url:
def procesar_puestos_para_url(lista_puestos_y_seniorities):
  puestos_para_url = []
  for puesto in lista_puestos_y_seniorities:
    puesto = puesto.replace(" ","-")
    puesto = puesto.lower()
    puestos_para_url.append(puesto)
  return puestos_para_url

puestos_para_url = procesar_puestos_para_url(lista_puestos_y_seniorities)

puestos_para_url

#El criterio que tomaré para definir jr, ssr, y sr es el siguiente:
# Jr = 0-1 año exp, Ssr = 2-4 años exp, Sr = 4+ años exp
anos_exp = ["0 a 1 año exp", "2-4 años exp", "4+ años exp"]

['junior Technical Leader', 'Technical Leader', 'senior Technical Leader', 'junior Project Leader', 'Project Leader', 'senior Project Leader', 'junior Project Manager', 'Project Manager', 'senior Project Manager', 'junior Product Owner', 'Product Owner', 'senior Product Owner', 'junior Data Architect', 'Data Architect', 'senior Data Architect', 'junior Software Architect', 'Software Architect', 'senior Software Architect', 'junior Scrum Master', 'Scrum Master', 'senior Scrum Master', 'junior Machine Learning Engineer', 'Machine Learning Engineer', 'senior Machine Learning Engineer', 'junior Data Scientist', 'Data Scientist', 'senior Data Scientist', 'junior Data Engineer', 'Data Engineer', 'senior Data Engineer', 'junior Data Analyst', 'Data Analyst', 'senior Data Analyst', 'junior Database Administrator', 'Database Administrator', 'senior Database Administrator', 'junior UX', 'UX', 'senior UX', 'junior Javascript Developer', 'Javascript Developer', 'senior Javascript Developer', 'juni

In [ ]:
def generar_urls_para_scrapear_sueldos(puestos_para_url):
  urls_a_scrapear = []
  for puesto in puestos_para_url:
    url = "https://www.glassdoor.com.ar/Sueldos/"+puesto+"-sueldo-SRCH_KO0,"+str(len(puesto))+".htm?clickSource=searchBtn"
    urls_a_scrapear.append(url)
  return urls_a_scrapear

urls_a_scrapear = generar_urls_para_scrapear_sueldos(puestos_para_url)

urls_a_scrapear

['https://www.glassdoor.com.ar/Sueldos/junior-technical-leader-sueldo-SRCH_KO0,23.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/technical-leader-sueldo-SRCH_KO0,16.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/senior-technical-leader-sueldo-SRCH_KO0,23.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/junior-project-leader-sueldo-SRCH_KO0,21.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/project-leader-sueldo-SRCH_KO0,14.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/senior-project-leader-sueldo-SRCH_KO0,21.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/junior-project-manager-sueldo-SRCH_KO0,22.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/project-manager-sueldo-SRCH_KO0,15.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/senior-project-manager-sueldo-SRCH_KO0,22.htm?clickSource=searchBtn',
 'https://www.glassdoor.com.ar/Sueldos/j

In [ ]:
"""# Extraigo info de una URL de ejemplo:
url_sueldo = urls_a_scrapear[0]
search_sueldo = requests.get(url_sueldo, verify = True, headers = {"User-Agent":'Mozilla/5.0'})
print(f'El status es: {search_sueldo.status_code}') # Chequeamos que haya salido todo bien
search_sueldo_parseada = bs(search_sueldo.content, 'html.parser')
print(search_sueldo_parseada.prettify())"""

'# Extraigo info de una URL de ejemplo:\nurl_sueldo = urls_a_scrapear[0]\nsearch_sueldo = requests.get(url_sueldo, verify = True, headers = {"User-Agent":\'Mozilla/5.0\'})\nprint(f\'El status es: {search_sueldo.status_code}\') # Chequeamos que haya salido todo bien\nsearch_sueldo_parseada = bs(search_sueldo.content, \'html.parser\')\nprint(search_sueldo_parseada.prettify())'

In [ ]:
"""#for i in range(20):
compania = str(search_sueldo_parseada.findAll(name='a', attrs = {'class' : 'css-f3vw95 e1aj7ssy3'}))
soup = bs(compania)
for t in soup.findAll('a'):
  compania = t.string
  print(compania)"""

"#for i in range(20):\ncompania = str(search_sueldo_parseada.findAll(name='a', attrs = {'class' : 'css-f3vw95 e1aj7ssy3'}))\nsoup = bs(compania)\nfor t in soup.findAll('a'):\n  compania = t.string\n  print(compania)"

In [ ]:
"""for i in range(20):
  puesto = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-job-title'})) # En data-test, item-n cambia según el aviso. Vamos a tener que iterar en estos
  soup = bs(puesto)
  for t in soup.findAll('span'):
    puesto = t.string
    print(puesto)"""

"for i in range(20):\n  puesto = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-job-title'})) # En data-test, item-n cambia según el aviso. Vamos a tener que iterar en estos\n  soup = bs(puesto)\n  for t in soup.findAll('span'):\n    puesto = t.string\n    print(puesto)"

In [ ]:
"""for i in range(20):
  sueldo_promedio = str(search_sueldo_parseada.findAll(name='div', attrs = {'data-test' : f'salaries-list-item-{i}-salary-info'}))
  soup = bs(sueldo_promedio)
  soup
  for t in soup.findAll('h3'):
    sueldo_promedio = t.string
    print(sueldo_promedio)"""

"for i in range(20):\n  sueldo_promedio = str(search_sueldo_parseada.findAll(name='div', attrs = {'data-test' : f'salaries-list-item-{i}-salary-info'}))\n  soup = bs(sueldo_promedio)\n  soup\n  for t in soup.findAll('h3'):\n    sueldo_promedio = t.string\n    print(sueldo_promedio)"

In [ ]:
"""for i in range(20):
    rango = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-salary-range'}))
    soup = bs(rango)
    if(rango != "[]"):
        for a in soup.findAll('span'):
          rango = a.get_text()
    else:
      rango = str(search_sueldo_parseada.findAll(name='div', attrs = {'data-test' : f'salaries-list-item-{i}-salary-info'}))
      soup = bs(rango)
      for a in soup.findAll('h3'):
        rango = a.get_text()"""


'for i in range(20):\n    rango = str(search_sueldo_parseada.findAll(name=\'span\', attrs = {\'data-test\' : f\'salaries-list-item-{i}-salary-range\'}))\n    soup = bs(rango)\n    if(rango != "[]"):\n        for a in soup.findAll(\'span\'):\n          rango = a.get_text()\n    else:\n      rango = str(search_sueldo_parseada.findAll(name=\'div\', attrs = {\'data-test\' : f\'salaries-list-item-{i}-salary-info\'}))\n      soup = bs(rango)\n      for a in soup.findAll(\'h3\'):\n        rango = a.get_text()'

In [ ]:
"""for i in range(20):
  num_sueldos = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-salary-count'}))
  soup = bs(num_sueldos)
  for a in soup.findAll('span'):
    num_sueldos = a.string
    print(num_sueldos)"""

"for i in range(20):\n  num_sueldos = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-salary-count'}))\n  soup = bs(num_sueldos)\n  for a in soup.findAll('span'):\n    num_sueldos = a.string\n    print(num_sueldos)"

In [ ]:
#Creamos el dataframe con el que trabajaremos al final
dataframe = pd.DataFrame(columns = ["Company","Position","Average Salary (ARS/month)","Salary Range (ARS/month)","Number of salaries informed", "URL"])
drive.mount('/content/drive')

def saca_info_sueldos(url_puesto,n_items): #Vamos a sacar la info de los primeros 20 puestos para cada página, por lo que idealmente n_items quedará en 20
  # Extraigo info de una URL de ejemplo:
  search_sueldo = requests.get(url_puesto, verify = True, headers = {"User-Agent":'Mozilla/5.0'})
  time.sleep(3*np.random.random())
  print(f'El status es: {search_sueldo.status_code}') # Chequeamos que haya salido todo bien
  search_sueldo_parseada = bs(search_sueldo.content, 'html.parser')

  # COMPAÑIAS
  lista_companias = []
  compania = str(search_sueldo_parseada.findAll(name='a', attrs = {'class' : 'css-f3vw95 e1aj7ssy3'}))
  soup = bs(compania)
  for t in soup.findAll('a'):
      compania = t.string
      lista_companias.append(compania)
  
  # PUESTOS
  lista_puestos = []
  for i in range(n_items):
      puesto = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-job-title'})) # En data-test, item-n cambia según el aviso. Vamos a tener que iterar en estos
      soup = bs(puesto)
      for t in soup.findAll('span'):
        puesto = t.string
        lista_puestos.append(puesto)

  # SALARIO PROMEDIO
  lista_salarios_promedio = []
  for i in range(n_items):
    sueldo_promedio = str(search_sueldo_parseada.findAll(name='div', attrs = {'data-test' : f'salaries-list-item-{i}-salary-info'}))
    soup = bs(sueldo_promedio)
    for t in soup.findAll('h3'):
      sueldo_promedio = t.string
      lista_salarios_promedio.append(sueldo_promedio)
  
  # RANGOS SALARIALES
  lista_rangos = []
  for i in range(n_items):
    rango = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-salary-range'}))
    soup = bs(rango)
    if(rango != "[]"):
      for a in soup.findAll('span'):
        rango = a.get_text()
        lista_rangos.append(rango)
    else:
      rango = str(search_sueldo_parseada.findAll(name='div', attrs = {'data-test' : f'salaries-list-item-{i}-salary-info'}))
      soup = bs(rango)
      for a in soup.findAll('h3'):
        rango = a.get_text()
        lista_rangos.append(rango)
  
  # NÚMERO DE SUELDOS
  lista_num_sueldos = []
  for i in range(n_items):
      num_sueldos = str(search_sueldo_parseada.findAll(name='span', attrs = {'data-test' : f'salaries-list-item-{i}-salary-count'}))
      soup = bs(num_sueldos)
      for a in soup.findAll('span'):
        num_sueldos = a.string
        lista_num_sueldos.append(num_sueldos)

  urls = [url_puesto] * n_items #Agrego la url de la búsqueda

  dictionary = {
    'Company': pd.Series(lista_companias), 
    'Position': pd.Series(lista_puestos), 
    'Average Salary (ARS/month)': pd.Series(lista_salarios_promedio),
    'Salary Range (ARS/month)': pd.Series(lista_rangos),
    'Number of salaries informed': pd.Series(lista_num_sueldos),
    'URL' : pd.Series(urls)
    }

  """print(len(lista_companias))
  print(len(lista_puestos))
  print(len(lista_salarios_promedio))
  print(len(lista_rangos))
  print(len(lista_num_sueldos))"""


  df = pd.DataFrame(dictionary)
  """frames = [dataframe,df]
  df_temp = pd.concat(frames)"""

  """#Guardo el dataframe a cada rato por las dudas que se me corte el scrapeo
  path = '/content/drive/My Drive/scraping_glassdoor_argentina.csv'

  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df_temp.to_csv(f)"""

  return df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
frames = []
final = pd.DataFrame(columns = ["Company","Position","Average Salary (ARS/month)","Salary Range (ARS/month)","Number of salaries informed", "URL"])
for url in urls_a_scrapear:
  frames.append(saca_info_sueldos(url,20))
  time.sleep(1*np.random.random())
  final = pd.concat(frames,ignore_index=True)
  #Guardo el dataframe a cada rato por las dudas que se me corte el scrapeo
  path = '/content/drive/My Drive/scraping_glassdoor_argentina.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    final.to_csv(f)
final

El status es: 200
El status es: 200
El status es: 200
El status es: 200


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: Deprecation

El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status es: 200
El status 

,Company,Position,Average Salary (ARS/month),Salary Range (ARS/month),Number of salaries informed,URL
0,Nielsen,Technical Analyst - Por mes,$ 96.119,Rango: $74 K - $105 K,16 sueldos,https://www.glassdoor.com.ar/Sueldos/junior-te...
1,Globant,Technical Manager - Por mes,$ 319.762,Rango: $167 K - $448 K,9 sueldos,https://www.glassdoor.com.ar/Sueldos/junior-te...
2,Avature,Technical Support Specialist - Por mes,$ 105.870,Rango: $90 K - $134 K,6 sueldos,https://www.glassdoor.com.ar/Sueldos/junior-te...
3,Kin + Carta,Technical Analyst - Por mes,$ 149.286,Rango: $135 K - $228 K,5 sueldos,https://www.glassdoor.com.ar/Sueldos/junior-te...
4,Globant,Technical Lead - Por mes,$ 166.928,Rango: $138 K - $211 K,4 sueldos,https://www.glassdoor.com.ar/Sueldos/junior-te...
...,...,...,...,...,...,...
4975,Globant,UX Designer - Por mes,$ 131.408,Rango: $105 K - $192 K,17 sueldos,https://www.glassdoor.com.ar/Sueldos/senior-di...
4976,Freelancer,Diseñador Gráfico - Por mes,$ 64.037,Rango: $30 K - $102 K,15 sueldos,https://www.glassdoor.com.ar/Sueldos/senior-di...
4977,Gobierno de la Ciudad de Buenos Aires,Diseñador Gráfico - Por mes,$ 81.413,Rango: $67 K - $115 K,12 sueldos,https://www.glassdoor.com.ar/Sueldos/senior-di...
4978,Diseñadora Gráfica,Diseñador Gráfico - Por mes,$ 62.043,Rango: $23 K - $102 K,11 sueldos,https://www.glassdoor.com.ar/Sueldos/senior-di...
